# Feature Engineering

## Applying transformations to test data

To avoid data leakage, it is important to not make any decisions (e.g. scaling, outlier cutoffs) based on test data. If transformations are necessary, the criteria should be the same as that used on the training data.

In general, changes to the form and scale of data, including filling missing values, needs to be performed on test data as well. Outliers should usually be left: the model needs to be able to deal with them, they are just removed from training because they can skew results reducing generalizability.

## Encoding categories

### Dummies vs. one-hot encoding

Both convert categories into a set of features encoded with 1 or 0. The technical difference is that one-hot encoding creates a feature for each category, while dummy encoding includes all but one features, which serves as a reference. The latter is necessary for linear models to avoid collinearity and results in the intercept capturing the average for reference categories.

### Dealing with large numbers of categories

1. Encode only a limited number of the largest categories, combining the rest into a reference "other" group.
1. Encode the categories by values on the target features (needs to be based on only on training data).

## Missing data

1. Observations with missing values can be dropped if they are expected to be random and not too relatively common.
1. In categorical data, a "missing" category can often be meaningful.
1. For continuous features, it is common to use a measure of central tendency like median or mean. Note that it is good practice to calculate variance, if needed, *prior* to filling missing values in this way. Also, when missing values are calculated from data, this should be based only on training data and then the same values should be used in test data.

## Outliers

Extreme values can skew models.
1. Remove a fixed share of extreme values, e.g. 5%
1. Define a cutoff, e.g. 3 SDs

## Transforming continuous data

### Rescaling
1. Standardization: centered at mean, with standard deviation as unit
1. Min-max scaling: shifted and compressed into range from 0 to 1

### Reshaping
1. Log transformation: useful for approximating a normad distribution with data skewed towards 0. Data must first be made strictly positive before transformation.
1. Scikitlearn implements a wider family of power transformations that use an estimable parameter to approximate normal distributions and can handle negative values.

## Text data

### Tokenization

Blocks of text need to be broken up into analyzable units or "tokens."

1. "Word tokenization" refers to breaking the text into a list of words.
1. Non-alphabetic characters (e.g. punctuation) can be treated as tokens in their own right, or disregarded (e.g. replacing with `''` any characters captured with the regex `[^a-zA-z]`). Note the need to consider case-sensitivity.
1. Simple tokenization treats "apple" and "apples" as distinct. **Stemming** attempts to convert derived word forms to their base. NB this is very difficult in English because there are so many cases where "endings" are just part words (e.g. `ceiling`) or conjugation changes the "base" (e.g. `give` -> `gave`). **Lemmatization** refers to more elaborate techniques are mapping related words to a root word.
1. **Stop words** refer to common words that serve a grammatical function without conveying much in the way of distinctive meaning, like articles and some prepositions. It is common to drop them as tokens.
1. **Part of speech tagging** seeks to identify the grammatical function of each word from it
1. **Chunking** and **parsing** break sentences into meaningful units. One common example is the identification of named entities. Full sentence parsing breaks it down into a grammar tree.
1. It is often useful to store numeric summaries of text, e.g. number of characters, number of words, and characters per word

### Vectorization

Textual data needs to be transformed into data-points, i.e. feature vectors, to be used in machine learning. There are various ways to do this:
1. **Count vectorization**, or "bag of words": indicating or counting the appearances of words in each text, so that there is one feature for each words in the dictionary. Refinements include excluding too-common "stop words" that do not convey much information, or not encoding words that appear only rarely.
1. **Term frequency-inverse document frequency** (TF-IDF): scales the value for a given observation by its relative frequency in that text and in the set of texts. In other words, it measures how relatively frequent the word is in a given text versus other texts. $$ \text{TF-IDF} = \frac {\text{count of this word in doc}} {\text {total words in doc}} \cdot \log \frac{\text{total number of docs}} {\text{count of docs containing this word}} $$ Though note that scikitlearn uses a smoothed variant that adds 1s to numerator, denominator and log of IDF, and then Euclidean normalized each row.
1. **N-grams**: bag of words methods (including TF-IDF) do not capture word order or sequence in any way. This can be addressed by encoding not individual words but instead unique sequences of some length (e.g. bigrams). Note the resource-intensity of this, producing $|\text{vocbaulary}|^n$ features to encode $n$-grams.

Note that vectorization, like any feature transformation, represents some kind of information input to the model, so it should be based only on training data.

The `NLTK` package implements many tools.

## Images

### Raw pixel vectors

Images can be represented as flattened vectors, where each element represents a pixel (either as a brightness or an RGB code). This can work for some applications, but its fundamental problem is that it cannot effectively handle variations in position and orientation.

Convolutional neural networks address this by transforming pixel vectors with "patches" rather than full weight vectors. 

### Interpreting shapes

While convolutional neural networks (given an architecture, which includes the size of patches and number of layers) will fit a transformation to a learning task, there are also various more structured procedures that have been developed to develop more interpretable data from images.

Many of these involve identifying changes in pixel values that correspond to meaningful image elements. **Edge identification** finds differences in pixel values thus highlighting the borders of objects. **Oriented gradients** compare the magnitudes of shifts around a pixel in the X and Y directions. **HOG** features (histogram of oriented gradients) counts the magnitude-weighted distribution of orientations within tiles of fixed size.

## Time series data

With multiple time series, they need to be combined in some way, either aggregating away the time dimension or averaging across observations for each time point. In the former case, the features will be the different series, in the latter, the time points.

### Filtering and aggregating

Before taking aggregates like mean or median or, especially, variance, it often makes sense to filter out noise. This can be done, e.g., with a rolling average.

Periodic time-series data (like audio) can be decomposed into a combination of waves of different frequencies. A **spectrogram** uses a heatmap to represent the weights of different-frequency waves in Fourier transforms calculated on a moving window across the time series. Features such as centroid and bandwidth can then be extracted from spectrograms.

# Supervised Learning

## Logistic Regression

Logistic regression predicts the log odds of a binary outcome dependent on other factors.

Some definitions:
1. Given a probability $p$ of an outcome, the odds of that outcome are $\frac{p}{1-p}$. Note that with an observed set of outcomes, because of the shared denominator, this is equivalent to $\frac{f^+}{f^-}$, where $f^+$, where $f^+$ is the number of cases with a positive outcome and $f^-$ the number of cases with a negative outcome.
1. The relative likelihood of an event for two different individuals can be summarized with the odds ratio: $\frac {\text{odds}_a}{odds}_b =\frac{p_a}{1-p_a} (\frac{p_b}{1-p_b})^{-1} = \frac{(1-p_b)p_a}{(1-p_a)p_b} = \frac {f_b^-f_a^+}{f_a^-f_b^+} $. 
1. For a $2 \times 2$ frequency table, this is easy to calculate:

In [1]:
import pandas as pd

two_way_table = pd.DataFrame(
    {'y+': [6, 162], 'y-': [13, 2343]}, index=['x+', 'x-']
)
two_way_table['odds'] = two_way_table['y+']/two_way_table['y-']
print(two_way_table)
print("Odds ratio:", two_way_table.loc['x+', 'odds']/two_way_table.loc['x-', 'odds'])

     y+    y-      odds
x+    6    13  0.461538
x-  162  2343  0.069142
Odds ratio: 6.6752136752136755


Logistic regression allows us to estimate the impacts of a vector of variables $X$ using MLE.
$$ \text{logit}(p) = \log \frac{p}{1-p} = \beta_0 + \beta X $$

Once estimated as $\hat y = \hat \beta_0 + \hat \beta X$, $e^{\hat y}$ for a new $X'$ is the estimated odds of the outcome.

For an independent variable of interest $X_j$, the coefficient $\beta_j$ represents the change in the log odds associated with changing it (by 1 unit if continuous, from negative to positive if binary). Let $\hat y=\log \frac{p}{1-p}$ be the base predicted log odds, and $\hat y'=\log \frac{p'}{1-p'}$ the new prediction, then: $\beta_1 = \hat y' - \hat y$. With logarithm arithmetic, this is equivalent to the log odds ratio associated with the change in the ration. Thus, to report the odds ratio associated with a change in one of the variables, all else equal, we need only calculate $e^{\hat \beta_j}$.

## Regularization

A problem faced by "big data" is that the available data tends to be "wide" rather than "tall": there are lots of measures relative to the number of observations. This is the inverse of the classical assumption of statistics. Regression performs best when data is clustered around a meaningful "middle." Various methods of regularization (or "penalized regression") seek to counterbalance this. In these methods, what is minimized is not the sum of squared residuals alone but a cost function combining the error with some penalty for overfitting. 
1. Ridge regression ($l_2$ penalty): the penalty term is the square sum of the coefficients $\lambda \sum B_j^2$. This is equivalent to the square length of the parameter vector.
1. Lasso ($l_1 penalty$): the sum of the absolute values of the coefficients $\lambda \sum |B_j|$. This is equivalent to the manhattan length of the parameter vector. This has the effect of pushing parameter vectors onto the "axes", whereas the penalty rate of ridge regression increases with the magnitude of coefficients and reduces parameters *proportionately*. This results in a "sparser" parameter vector, which is useful insofar as it allows for reducing the number of relevant features.

The mathematical logic for this can be linked back to Stein's paradox, which shows that even when estimating parameters for a set independent distributions, it is better to "shrink" the estimates towards the grand mean of the data rather than using only the observations for each type.

## Decision Trees and Ensemble Methods

A decision tree classifies observations through a series of binary decision points (e.g. $X_1\le0$). A decision tree is fit by recursively considering, for a group of point, which split on which single feature would result in two groups with highest weighted average "purity" in terms of labels. Purity (or, in practice, its absence) can be measured in different ways, including entropy ($-\sum_j p_j \log p_j$) and Gini impurity ($1-\sum_j p_j^2$), where $p_j$ is the share of a label $j$ in a cell.

When compared to linear methods like Perceptron or Logistic Regression, the main advantages of Decision Trees are:
1. Fast fitting and prediction
1. Indifference to scale of features and to numerical vs. categorical features
1. Interpretability of decision points (as opposed to $\theta$ vectors)

The main disadvantages are:
1. All decision boundaries are "orthogonal," based on the value of one feature at a time, as opposed to the diagonal or even interacted relationships captures by linear models.
1. Strong tendency towards over-fitting.

That is to say, with unlimited depth, a decision tree can always perfectly classify training data, though this can result in a very high-variance classifier (i.e. with lots of very small cells). To improve generalizability, it can help to:
1. Limit the depth of the tree.
1. Limit the minimal size of a leaf.
1. Fit multiple trees and select the majority result (an example of ensemble learning known as a **random forest** in its full form). In general, this is a useful technique for high-variance models.

With mixed leaves or multiple trees, the model can either output a maximum likelihood label or probabilities. Decision trees or random forests can also be adapted to regression problems by outputting the average of values placed in a leaf based on minimizing some loss function.

### Random Forest

A random forest needs to introduces some randomness (since the naive fitting algorithm is deterministic), including:
1. Bootstrap resampling (with replacement) the data for each tree (a technique known as **bagging**)
1. Considering only a random subset of features (as a rule of thumb, $\sqrt K$ of them) for each decision point (or for each tree? different descriptions are ambiguous)

Random Forests can achieve classification performance in the ballpark of much more complicated neural networks, but are much easier to set up and even parallelize because of the independence of the models.

**Bagging** as a method for ensemble learning can be used for other kinds of models as well. It can be augmented also by assigning weights to each model.

### Boosting

Boosting is an iterative ensemble method. It does not just average different models but also learns weights for the observations and models. Generally speaking, *misclassified* cases in a given modeling iteration are giving greater weight in next model fitting, while models are weighted inversely to their error. Two key hyperparameters are the complexity of the trees (often a single-split "stump" works surprisingly well) and the "learening rate" response for the weights.

For example **Adaptive Boosting** (AdaBoost) for binary classification begins with equal weights for each observation. It fits a classifier and calculates a weighted average error rate. The weight for the model $a_m = \log \frac {1-\text{error}_m} {\text {error}_m}$ and re-weights the observations as $w_i' = w_i e^{a_m (1-z_i)}$, where $z_i$ is 1 for a correct prediction, 0 otherwise. The output of the ensemble is then just the sign of the weighted average of model predictions.

**Gradient Boosting** similarly sequentially fits models, but instead of weighting the observations, each model after the first fit the *residuals* ($y- \hat y$) of the previous model. The final prediction is then the sum of the predictions of all of the models, often with a decaying rate of influence.

Both techniques are implemented in `sklearn.ensemble` as the model classes `AdaBoostClassifier`, `AdaBoostRegressor`, `GradientBoostingClassifier`, and `GradientBoostingRegressor`. Note that AdaBoost by default is based on decision trees but any model can be passed as `based_estimator`. while gradient boosting is implemented only with decision trees.

Boosting methods are powerful, but they can overfit as well because they "chase" hard-to-fit cases. Compared to random forest, boosting is slower because it must be fit iteratively rather than in parallel, and it is more sensitive to its hyperparameters. On the other hand, boosting methods have an advantage in interpretability by outputting feature importances.



## Metrics

### Classification

For binary labels:
1. **Confusion matrix**: splits predictions into $2\times2$ table, where (at least in scikit learn implementations) rows are reality (e.g. negative, positive) and columns are prediction. Many metrics represent different combinations of these numbers, based on which kinds of correct and incorrect predictions are most important practically.
1. **Accuracy**: the share of predictions that are correct. The main downside of this is that its significance depends heavily on baseline right.
1. **Precision**: the share of positive predictions that are accurate. Favors avoidance of false positives.
1. **Recall**: the share of positive cases that are correctly predicted. Favors avoidance of false negatives. This is also called **sensitivity**, and the analogous measure for negative labels is called **specificity**.
1. **F1 Score**: harmonic mean of precision and recall, i.e. $2 \frac{(\text{precision}) (\text{recall})}{\text{precision} + \text{recall}}$, or the the true positive predictions over the sum of the true positive predictions and the average of the two kinds of false predictions. If labels are imbalanced, this often works better when it is the positives that are rarer (otherwise the score will have a high baseline rate).
1. **Matthews Correlation Coefficient** (or phi coefficient): basically a measure of how much of the data is in the positive diagonal of the confusion matrix. It is calculated as the differences of the products of the positive and negative diagonals, scaled to the square root of the product of all four marginal totals. The advantage of this is that it simultaneously takes into account positive and negative predictions, and so does not depend on labeling (as the others do). With highly imbalanced data, even F1 score can be very different when labels reversed (in extreme case, where there are no predictions in one class).

For predicted probabilities:
1. **(Area under the ) ROC curve**: plot of true positive rate against false positive rate based on different probability tresholds. This can be sumamrized by the area under the curve (in effect, a fraction of 1). Note, like accuracy, this is sensitive to imbalanced data.
1. **(Area under the) Precision-recall curve**: plot of precision against recall based on different probability tresholds. AUC can also be used as summary. With imbalanced data, this can be sensitive to coding.
1. **Log loss**: a cost function that can be used a metric. It has the effect of penalizing wrong predictions more if they are more confident.

For multi-class problems:
1. The confusion matrix can be extended to any number of rows and columns.
1. Precision, recall, and F1 score can be calculated as average of 1-against-all metrics for each label. This average can be taken on counts (i.e. sum of true positive predictions over sum of positive predictions for all labels), rates, or weighted rates (weighting by real label count). These are called `micro`, `macro`, and `weighted` in scikit-learn's syntax. These different averages will be close if the categories are relatively balanced, otherwise they will differ based on how they treat smaller classes. (I think `micro` and `weighted` are the same for recall because in either case, the total true size of each class is in the denominator.)
1. Log loss can also be generalized to multi-class case, which basically averages over the probabilities assigned to the true labels for each case.

### Regression

1. **Mean absolute error (MAE)**: the average of the absolute value of the residuals. Note that compared to MSE, this gives less weight to outliers.
1. **Mean square error (MSE)**: the average of the squared residuals. The lower the better. Note that this measure is in the unit of the dependent variable, squared (this can be address by taking the square root, i.e. RMSE), and will depend on the scaling of that variable.
1. $R^2$: the share of variance in the dependent variable that is "explained" by the model, i.e. variance minus MSE, over variance. The maximum is 1, but the minimum is technically $-\infty$ if the model actually manages to be worse than just guessing the mean. While $R^2$ is the default metric for explanatory problems, it doesn't really capture the accuracy of predictions in intuitive terms. For this, MAE and RMSE can be better, as representations of "how far away the prediction is, on average.
1. **Adjusted** $R^2$: a formula that penalizes the reduction of degrees of freedom by the introduction of additional predictive features. It does this by scaling the MSE *up* by the ratio or sample size to degrees of freedom. That is to say, where $n$ is the number of observations and $k$ is the number of predictors, $R^2_{adj}=1- \frac{(1-R^2)(n-1)}{n-k-1}$.
1. **Root Mean Squared Logarithmic Error (RMSLE)**: the square root of the average of the squared logarithm of the ratio of true to predicted values. This penalizes errors based on their proportional magnitude. Note that because it puts the prediction in the denominator, under-prediction is more penalize than over-prediction.
1. **(Log) Likelihood**: the joint probability of producing the data given the estimated model. With independent observations, this is the product of the probabilities of each observation, and so it is common to calculate the log likelihood, allowing for calculation of sums instead of products.
1. **Bayes Information Criterion (BIC)**: a measure that balances the likelihood against the number of parameters. With $n$ as the number of observations, $k$ as the number of parameters (including the constant and the error terms), and $L=p(x|\hat \theta)$, $\text{BIC}=k \ln(n)-2\ln(L)$.
1. **Akaike Information Criterion (AIC)**: an alternative (slightly older) measure in the same line of thought as the BIC. $\text{AIC}=2k-2\ln(L)$. Note that for $n>e^2$, the AIC penality for additional parameters is less than for the BIC.

# Time Series Analysis

## Properties of time series

The property of a time series that requires a special approach is **memory**. That is to say that the value at $t$ in some way depends on the value at $t-1$. This is also known as **auto-correlation**.

Time series analysis is structured around a mathematical decomposition of an observation into **signal** and **noise**. These are said to be **stationary** (or not) if they have constant mean, standard deviation, and autocorrelation (or not). They have **seasonality** if these vary at regular periods.

Pre-processing (or filtering) can seek to:
1. De-trend the data (if non-stationary or seasonal)
1. Account for autocorrelation
1. Smoothing out noise

### Stationarity

For example, if we are interested in the seasonality of some time series, we can fit a linear model over time (or log-linear for exponential growth) and subtract the prediction, i.e. look at the residuals. Or a trend can be estimated just with a rolling average (e.g. in Pandas, using `rolling(window).mean()`).

A time series is **trend stationary** if the data is stationary once the trend is removed. It is **difference stationary** if, for example, the first differences ($y_t - y_{t-1}$) show a stable distribution. For example, stock values often do not have a stable trend but are difference stationary.

If the first difference is entirely random with no auto-correlation, then the series can be desrcibed as a **random walk model** $y_t = y_{t-1} + w_t$, where $w_t$ is an i.i.d. noise term. A random walk with drift is a model in which the differences have a non-zero mean: $y_t = \mu + y_{t-1} + w_t$. In a random walk, the autocorrelation is 1, or equivalently, the correlation between the difference and the previous-period value is 0: the Dicky-Fuller test uses this as a null hypothesis. The Augmented Dicky Fuller Test extends this to multiple lags and represents a test of whether the process has a unit root.

Non-stationarity in variance can be address with tranformations like log, square root, or proportion

Sometimes, although two time series are each separately random walks, the linear relationship between them is not. This entails modeling the spread between two variables, for example prices of substitutes. Thus,
$$P_t = \mu+cQ_t+\epsilon_t$$
For cointegreated variables, the Dickey-Fuller test of $P_t - cQ_t$ should reject the null hypothesis.

Seasonality can be removed by fitting periodic averages and subtracting these. In general, modeling time series entails experimenting with differences and seasonal differences in order to identify a transformation that yield stationarity. This is important, for example, because any two time series variables that have trends or similar seasonality will be correlated, even if totally unrelated in reality.

### Auto-correlation

Auto-correlation is defined using the same formula as correlation, but instead of two variables, it is between one variable and that same variable lagged by some constant (here, $k$):
$$ r_k = \frac {\sum_{i=1}^{N-k} (Y_i - \bar Y)(Y_{i+k} - \bar Y)}{\sigma^2_Y} $$
Auto-correlation can both capture periodicity and memory. The former looks like a high auto-correlation at some specific period, while the latter looks like a gradually decreasing auto-correlation over greater periods. The **autocorrelation function** shows the autocorrelation at all lags, while the **partical autocorrelation function** shows the unique correlation with past lages. In general, an `AR(p)` shows a clear cutoff only on the PACF after `p` lags, and a `MA(q)` shows a clear cutoff after `q` lags on the ACF. Trends should be removed *before* looking at autocorrelation: very high and consistent autocorrelation reflects non-stationarity, and data should be differenced.

Seasonality appears in ACF as peaks at lags greater than 1.

### Smoothing

Smoothing strategies include low-pass filtering or statistical filters such as weighted average of nearby values. One issue with smoothing is that it loses information at ends of series. Generally speaking, the point of smoothing is to preserve larger, less frequent events while removing smaller, more frequent changes.

## Time series models

### Auto-regressive models

An auto-regressive model of order $p$, or $\text{AR}(p)$, represents an observation as a function of $p$ earlier observations (plus a drift term and noise). So, an $\text{AR}(1)$ model is written as:
$$ X_t = \mu + \phi X_{t-1} + \epsilon_t $$
For the process to be stationary, $-1<\phi<1$, representing of decay of the autocorrelation function. $\phi=1$ represents a random walk, $\phi=0$ represents random noise. Positive and negative values of $\phi$ reflect *momentum* and *mean reversion*, respectively.

The Partial Autocorrelation Function shows the $\phi$ values for increasing orders of AR models. Generally, the order of the AR model fitted should be the last for which the coefficient is significantly different from 0. Alternatively, we can look at the BIC of various orders of models to choose the one with the lowest value.

### Moving-average models

A moving average model of order $q$, or $\text{MA}(q)$, represents an observation as a function of $q$ previous error terms (plus drift and noise). So, an $\text{MA}(1)$ model is written as:
$$ X_t = \mu + \epsilon_t + \theta \epsilon_{t-1} $$
An MA model is always stationary. The 1-period autocorrelation for this model would be $\frac {\theta}{1+\theta^2}$ and 0 for longer lags. A consequence of this is that the (MLE) forecast for an MA model beyond $q$ steps is always $\mu$.

Note that it is possible to represents an AR(1) model as a MA model of infinite order.

### ARIMA models

ARMA models assume stationarity. If the data is difference-stationary, then an ARMA model can be fit to differenced data, and predicted or forecasted levels reconstruction using cumulative sums. Alternatively, ARIMA models build-in the differencing to directly output predicted levels. (NOTE: in the Cowboy Cigarettes case study, the ARIMA(2,1,1) model yields very different predictions from the pre-differenced ARMA(2,1) model.

An autoregressive integrated moving average model of order $(p,d,q)$, where $p$ and $q$ are defined as above and $d$ is the number of differences to take. For example, an $\text{ARIMA}(0,1,0)$ model:
$$ X_t = \mu + X_{t-1} + \epsilon_t $$
is a random walk with drift. The value of $d$ should be sufficient to achieve stationarity of the data set.

### Seasonal models

Seasonality can be modeled by adding terms for autoregression ($P$), differencing ($D$), and moving averages ($Q$) discrete longer lags corresponding to season cycles (represented by $S$). These orders can be found by looking at the ACF and PACF (perhaps seasonal-differenced) at lags of multiples of the seasonal period.

### Models with eXogenous variables

Time-series models can be combined with linear modeling simply by adding other variables with their own coefficients.

### Model selection and evaluation

In addition to the heuristics mentioned above, models can be compared based on AIC and BIC, which weigh explanatory power vs. complexity. The BIC penalizes complexity more. For both, lower is better. They are accessible in `Results` objects through `summary()` and `aic` and `bic` attributes.

NB: some model orders will fail to fit because of stationarity constraints. Loops can use `try...except` structure.

For evaluation, model results store the 1-step-ahead prediction residuals in the `resid` attribute. These can be used to calculate MSE or MAE. The `plot_diagnostics()` method creates plots that show residuals: there should be no patterns and minimal deviations from standard normal. The `Q` and `JB` tests at the bottom of the `summary()` output use null hypotheses that residuals are uncorrelated and normally distributed, respectively. (Note that the Jaroqe-Bera test is rather sensitive to outliers.)

## Tools

### Statsmodels TSA

Tools for time series analysis are included in the `statsmodels` package's `tsa` module (`import statsmodels.tsa.api as tsa`). Note that the classes and methods below are directly visable in the API.
- `statsmodels.tsa.stattools.adfuller(x, maxlag=None, regression='c', autolag='AIC', store=False, regresults=False) -> (float, float, int, int, dict, float)`: Augmented Dickey Fuller Test, based on the null hypothesis that there is a unit root, i.e. non-stationary behavior (paradigmatically, a random walk). The 2nd return is the pvalue for the test. If the null cannot be rejected, the series will need to be differenced. Note that this does NOT test for stationarity in terms of variance.
- `statsmodels.tsa.stattools.coint(y0, y1, trend='c', method='aeg', maxlag=None, autolag='aic', return_results=None) -> float, float, dict` tests for cointegration between the two given variables. As with `adfuller`, this returns test statistics and p values for null hypothesis that there is no cointegration.
- `statsmodels.tsa.seasonal.seasonal_decompose(x, model='additive', filt=None, period=None, two_sided=True, extrapolate_trend=0) -> DecomposeResult`: decomposes a time series into `trend`, `seasonal` variation, and `resid`uals. These are accessible as attributes of returned object, along with the original data as `observed`. Included `plot()` method will put all of the elements on a line graph.
- `statsmodels.tsa.stattools.acf(x, adjusted=False, nlags=None, qstat=False, fft=True, alpha=None, bartlett_confint=True, missing='none')`: calculates the autocorrelation function for the given time series data. Returns a 1d array where the index is the number of lags (NB ACF(0) is always 1). Also, with `alpha` parameter, returns $1-\alpha$ confidences intervals. With `qstat=True`, returns Q statistics and p values for correlations. The `pacf()` estimates the partial autocorrelation function with the same syntax. There are also `tsa.graphics.plot_acf()` and `tsa.graphics.plot_pacf()` functions.
- `statsmodels.tsa.arima.model.ARIMA` and `statsmodels.tsa.arima.model.SARIMAX` implement ARIMA models with or without seasonality or exogenous regressors.
    - As with other `statsmodels` classes, the first two positional arguments in the constructor are the endogenous and exogenous variables. The former is the time series to be modeled, while the latter in a time series is optional.
    - Again as usual with `statsmodels` the `fit()` method on the model class returns a `Results` object with methods including `summary()`, `predict()` and (uniquely) `forecast(steps=1)`. Note that `predict()` by defaults gives predictions for input data; use `start` and `end` to adjust (including forecasting), which can take dates, date strings, or integers (indicating indices, negative to count from end). This by default (`dynamic=False`) makes each prediction based on previous values, to predict later values based on predictions, use `dynamic=True` or the point (index or date) with which to begin dynamic prediction. The `get_prediction()` and `get_forecast()` methods return a PredictionResults object, with attribute `predicted_mean` (a 1d array/series) and method `conf_int` (a $n \times 2$ array/frame).
    - Parameter `order=(p,d,q)` controls the non-seasonal parameters.
    - Parameter `seasonal_order=(P,D,Q,s)` controls seasonal structure, where `s` is the number of periods in the seasonal cycle.
- `statsmodels.tsa.arima_process.ArmaProcess(ar, ma)` creates an object for simulating an ARMA process. `ar` should be an array of autoregression coefficients, starting with $\mu$ and then giving the **negative** of the $\phi$ values. `ma` should be an array of moving average polynomial coefficients ($\theta$, not $-\theta$), use [1] for an AR model. The object .generate_sample(nsample) method simulates data.

### Automation with pmdarima
- The `pmdarima` package (`import pmdarima as pm`) implements automated model searching
- Basic syntax: `model = pm.auto_arima(y, X=None, m=1)`. Exogenous variables can be optionally included. Seasonal period needs to be specified as `m`; other order parameters are optimized algorithmically, though min and max values can be specified. This returns a *fitted* model object for the best order.
- Fitted models in `pmdarima` also have an `update(y)` method that will adjust parameters with new data.

### Visualization
- `matplotlib`
    - The `plot()` function will automatically plot a TimeDateIndex of a series as given as single positional argument
    - `fill_between(x, y1, y2)` will shade the area between `y1` and `y2` across the values gives in `x`.


# Unsupervised Learning

## K means

**See MIT ML notes.**

## Mean shift clustering

An alternative to K means that does not require setting $K$ as a hyperparameter or random initialization, instead depending (deterministically) on a window size. However, it is very computationally intensive.
1. For each point:
    1. Identify all points within the set window size
    1. Calculate the center of all of those points
    1. Repeat until center stabilizes
1. Group points based on the centers on which the algorithm converged for each

## Hierarchical clustering

A flexible technique for identifying commonalities among observations.
1. Calculate distances for all pairs
1. Repeat until no more grouping to be done:
    1. Identify two clusters with shortest distance between them and combine them:
        1. Set its position as the weighted average of the two
        1. Store the `height` of this combination as the distance between the two clusters

Hierachical clustering can be visualized as a dendrogram, and groups made by "cutting" it at some `height` value. Note that this algorithm can be done with different measures of distance, e.g.:
1. **single**: looks at the minimum distance between any two members of two clusters
1. **complete**: looks at the maximum distance between any two members of the two clusters
There is also a version that proceeds backwards through splits.

## Graph community detection

One way of thinking of networks is to say that a natural grouping maximizing the **modularity** of each group:
$$ M_m = \frac{1}{2L} \sum_{i,j \in C_m,i\ne j} (A_{ij}-\frac{k_ik_j}{2L}) $$
where $A_{ij}$ is the adjacency of two nodes (1 if connected, 0 if not), and $k_i$ is the number of edges connected to node $i$ (i.e. the sum of row $i$ in the adjaceny matrix $A$).

## T-distributed Stochastic Neighbor Embedding (t-SNE)

A method to visualize clustering high-dimensional data by projecting it onto a 2- or 3-dimensional space. Note that the transformed axes depend on (random) initialization as well as a hyperparameter `learning rate`. Implemeneted in `sklearn.manifold.TNSE`.

## Distance measures

Distance measures can be generalized using Minkowski metric:
$$ d_p(x, x') = (\sum_{k=1}^d |x_k - x_k'|^p)^{\frac{1}{p}}$$
Thus, with $p=1$, this yields Manhattan distance, with $p=2$, it is Euclidean distance.

## Assessing clusters

Judging what is a "good" clustering without labels is challenging. The "error" of clusters can be measured by comparing the points within them, but in principle, more clusters always reduces this. If we want to use the clusters for future data, there is a trade-off of "stability." One way to capture this is to segment data, fit the same algorithm with the same parameters to each, then compare the "decision boundary" or one with the "labels" of the other.

## Dimensionality Reduction

### PCA/SVM

High-dimensional data is often reducible because of correlation among dimensions. Principal Component Analysis uses orthogonal projection to preserve as much information as possible while reducing dimensionality.

With an $n$-dimensional vector space $V$ and an $m$-dimensional subspace $W$, each vector in $V$ can be decomposed as the sum of the orthgonal projection in $W$ and the component(s) orthogonal to $W$; the space of all such complements is called the orthogonal complement (which is an $n \times m$ dimensional subspace of $V$)

**The objective**: for a given, reduced number of dimensions, find orthonormal basis vectors $B$ (and associated scalars $\beta$) that minmize the square distance between the projected vector and the original vector, using grad optimization. The derivative of objective function with respect to each coefficient beta is $(-2/N)(X^T b - \beta)$, thus is at extreme where $\beta_i$ = $x^T_i b_j$ for observations $i$ and dimensions $j$. The difference between the reduced vector and the original is equal to the orthogonal complement: we need to reduce the square sum length of these complement vectors; substituting this back into objective loss equation and rearranging (using orthnormality of basis), we arrive at $B^T S B$, where $S$ is the covariance matrix of the data (assuming it has been normalized to mean = 0), which can further be reinterpreted as minimizing the "ignored variance" i.e. the sum of the variance of the data projected onto the complement subspace. Because basis vectors are constrained to be orthonormal, we can use lagrangian equation, which yields solution that it is the eigenvectors of the covariance matrix with the smallest eigenvalues that minimize the loss, and by extension that the principle subspace is defined by the eigenvectors in descending order of eigenvalues.

It is computationally and representationally useful to subtract by mean and divide by SD for each dimension; the resulting values are centered around 0 and unitless.

**The full algorithm**
1. Standardize data
1. Find eigenvalues and vectors
1. Find the covariance matrix
1. Take the vectors corresponding to the largest values to make principal subspace basis vectors, and then project the data

**Interpretation**
- PCA as an "autoencoder": the lower-dimensional scalars to the basis vectors is sometimes called the "code," insofar as the PCA process can be thought of as "encoding" the data and "decoding" it with the reconstruction; PCA a linear autoencoder, and there are alternatives using non-linear or deep-learning algorithms.
- PCA as lossy compression:	The lower-dimensional code can be thought of also as a compression of the data, which can approximate with "loss" of information the original

### Non-Negative Matrix Factorization

An alterantive dimensionality reduction algorithm that is more interpretable than PCA but requires that all values be nonnegative. Given a desired number of components $k$, NMF reduces the $d$-dimensional data $D$ to a $n \times k$ matrix $C$ and generates a $k \times d$ component matrix $B$ such that $D \approx C B$ while minimizing the l1 and l2 norms of the matrix.

The rows of the components matrix can be interpreted as "patterns" found in the data. E.g., with word frequency data, these will reflect collections of words that frequently appear together. With image data, whereas PCA tries to capture as much of the distinctiveness of the entire image in each component, NMF will extract distinct elements.

The reduced data matrix thus represents the "weight" of each pattern for each observation. Different observations can then be compared in light of the topic by cosine similarity, i.e. the cosine of the angle between two vectors: $\cos \theta = \frac {A \cdot B}{|A||B|}$ (i.e. the dot product between the normalized vectors).